## Take an existing WOD ragged array format file and create a new empty one ready for populating with new data

Bec Cowley, April 2023

In [1]:
import xarray as xr
import os
import netCDF4 as nc
import pandas as pd
from datetime import datetime as dt

In [2]:
# file name for our starting point
infile='/oa-decadal-climate/work/observations/WOD_CARS2022/2018/wod_ctd_2018.nc'
# output file that should have the same variables and no data included
outfile = '/oa-decadal-climate/work/observations/CARSv2_ancillary/outputNC/test.nc'

# Importing a list of global attributes

In [3]:
global_atts = pd.read_excel('/oa-decadal-climate/work/observations/CARSv2_ancillary/outputNC/cars_global_atts.xlsx', sheet_name='globals')

In [4]:
# take a look at our global attributes
global_atts

,Attribute,Value
0,institution,NaN
1,source,Data collated from institution global attribut...
2,references,World Ocean Database 2018. URL:http://data.nod...
3,title,CARSv2 multiple cast file
4,summary,Data for multiple casts from source global att...
5,id,NaN
6,naming_authority,gov.noaa.nodc
7,geospatial_lat_min,-75.29098
8,geospatial_lat_max,83.3485
9,geospatial_lat_resolution,point


In [16]:
from pathlib import Path
# now let's read in the csv CTD data from AIMS
AIMS_data_path ='/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/'
dir_contents = os.listdir(AIMS_data_path)

filelist = Path(AIMS_data_path).rglob('*.csv')
        

In [18]:
# read and append the data from each file
import csv

# TODO: assign the header values as well where possible. 
# Assign all variables. Following code only assigns lat/long/date

AIMS_latitude  = []
AIMS_longitude = []
AIMS_date      = []

for file in filelist:
    # file list is an object
    print(str(file))
    
    with open(str(file)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        found_date = False
        for row in csv_reader:
            #print(row)
            if row.count('LATITUDE:')>0:
                AIMS_latitude.append(float(row[1]))
            elif row.count('LONGITUDE:')>0:
                AIMS_longitude.append(float(row[1]))
            
            
            elif row.count('SAMPLE DATE:')>0: #row.count('SAMPLE DATE:')>0:
                found_date = True
                AIMS_date.append(pd.to_datetime(row[1]))
            line_count = line_count + 1
            if line_count > 20 and not found_date:
                AIMS_longitude.pop()
                AIMS_latitude.pop()
                break
                
            


/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV002.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV003.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV004.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV005.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV006.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-AUV007.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC001.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC002.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC003.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC004.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/tmp/ipykernel_4283/1575149944.py:27: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  AIMS_date.append(pd.to_datetime(row[1]))


/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC006.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC007.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC008.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC009.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC010.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC011.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC012.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC013.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC014.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-CSC015.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN055.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN056.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN057.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN059.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN060.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN061.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN062.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN063.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN064.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-INN065.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM079.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM080.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM081.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM084.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM085.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM086.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM087.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM088.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM089.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-KIM090.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

...

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP480.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP481.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP482.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP483.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP484.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP494.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP495.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP496.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP497.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP500.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP691.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP692.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP693.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP695.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP696.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP697.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP698.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP699.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP700.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP702.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP885.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP886.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP887.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP888.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP889.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP891.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP892.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP893.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP894.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQP895.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ134.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ136.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ137.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ138.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ139.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ140.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ141.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ143.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ144.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQQ145.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS012.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS013.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS014.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS015.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS016.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS017.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS018.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS019.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS020.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation-WQS021.csv
/oa-decadal-climate/work/observations/CARSv2_ancillary/AIMS/AimsWqCtdDataStation

In [19]:
AIMS_latitude

[-18.6283333,
 -18.4505,
 -18.4156667,
 -18.8145,
 -18.8153333,
 -18.895,
 -11.797,
 -11.6745,
 -11.8508333,
 -12.4566667,
 -13.4268333,
 -13.9793333,
 -14.3243333,
 -14.7746667,
 -16.4821667,
 -16.5128333,
 -16.6848333,
 -21.0555833,
 -21.0576333,
 -21.1609333,
 -20.8848833,
 -21.20955,
 -21.1620833,
 -21.4736667,
 -21.6807833,
 -21.8083,
 -12.1855,
 -12.2325,
 -12.2245167,
 -12.33025,
 -12.42795,
 -12.4689167,
 -14.6683667,
 -14.9286667,
 -15.1821667,
 -15.5915667,
 -16.6256167,
 -19.8285,
 -21.417,
 -21.446,
 -21.4453333,
 -21.0593333,
 -21.06,
 -21.0563333,
 -21.3743333,
 -21.3743333,
 -13.9798,
 -13.16535,
 -11.4470167,
 -11.67065,
 -11.6137,
 -12.1377833,
 -12.2093167,
 -12.3385667,
 -12.7651,
 -12.9083333,
 -13.0807167,
 -13.1899167,
 -13.3446,
 -13.5272,
 -13.7699167,
 -14.1787167,
 -14.1403667,
 -16.2902833,
 -16.6860333,
 -16.6448167,
 -18.9352167,
 -20.8828833,
 -21.2383167,
 -21.0600833,
 -20.9963833,
 -21.0404667,
 -21.2336833,
 -21.3861667,
 -21.5336,
 -21.6965833,
 -18.6

In [20]:
# and make an identical outputfile with no data
with nc.Dataset(infile) as src, nc.Dataset(outfile,'w') as dst:
    # copy global attribute names
    dst.setncatts(src.__dict__)
    # try adjusting a couple of global attributes:
    dst.setncattr('institution', 'AIMS')
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(name, None)
    #copy all variables
    for name, variable in src.variables.items():
        x = dst.createVariable(name, variable.datatype, variable.dimensions)
        #and variable attributes
        dst[name].setncatts(src[name].__dict__)
    